In [1]:
import json
import pandas as pd
import requests
import sqlite3 as db
from sqlite3 import Error
from datetime import datetime

%run C:\Users\Joshua\Jupyter_Notebook_Folders\APIkeys.py
%run C:\Users\Joshua\Jupyter_Notebook_Folders\MacroData2\db_functions.py

In [2]:
db_name = 'MacroData2'

In [3]:
def get_BLS_data(seriesIDs, startyear, endyear):
    base_url = 'https://api.bls.gov/publicAPI/v2/timeseries/data/'  #this will not change
    headers = {'Content-type': 'application/json'}  #This will not changed !

    # For the key seriesid enter a list of series names you wish to download
    # For the key startyear enter the start year inside ""
    # For the key endyear enter the end year inside ""
    
    parameters = { 
        "seriesid":seriesIDs,
        "startyear":str(startyear), 
        "endyear":str(endyear),
        "catalog":True, 
        "calculations":False, 
        "annualaverage":False,
        "aspects":False,
        "registrationkey":os.environ['BLS_API_key'] 
     }

    data = json.dumps(parameters) # Converts the Python dictionary to JSON

    p = requests.post(base_url, data=data, headers=headers)
    json_data = json.loads(p.text)
    
    message = ""
    if json_data['message']:
        #message = "For series " + seriesIDs + ", no data for years: "
        for i in range(len(json_data['message'])):
            message += json_data['message'][i][-4:] + ", "
    
    return message, json_data 

In [4]:
def check_table_exists(tbl_name, db_name): 
    """
    Check if table exists
    """
    conn = create_connection(db_name)
    c = conn.cursor()
    exists = 0
    c.execute("SELECT count(name) FROM sqlite_master WHERE type='table'")
    if c.fetchone()[0] == 0:
        print("No tables in DB.")
    else:
        try:
            c.execute("SELECT COUNT(name) FROM sqlite_master WHERE type = 'table' AND name='" + tbl_name +"';")
        except Error as e:
            print(e)
        if c.fetchone()[0] == 1:
            exists = 1
        
    conn.close()
    
    return exists


def create_table(tbl_name, db_name):
    """
    create table
    """
    conn = create_connection(db_name)
    c = conn.cursor()
    
    sql_code1 = ("""CREATE TABLE IF NOT EXISTS """ + tbl_name + """ (
                    date PRIMARY KEY,
                ) WITHOUT ROWID;""")
    
    conn_execute(db_name, sql_code1)
    '''if not check_table_exists(tbl_name ,db_name):
        try: 
            input_df.to_sql(tbl_name, conn)
            print("Created ", tbl_name, " table.")
        except ValueError as e:
            print(e)
            print("Table was not added to the DB")'''    
    
    conn.close()
    
    
def add_data_to_db(df_EntryData, tbl_name, db_name):  
    """
    Add dataframe to 
    database: db_name 
    table: tbl_name
    """
    conn = create_connection(db_name)    
    try:
        df_EntryData.to_sql(tbl_name, con=conn, if_exists = 'append')
        print("Data added to DB: \n", df_EntryData)
    except ValueError as e:
        print(e)
        print("Data was not added to the DB")
    conn.close()
    

def table_to_df(tbl_name, db_name):
    conn = db.connect(db_name)
    try:
        df_fromDB = pd.read_sql_query("SELECT * FROM " + tbl_name + ";", conn, index_col="date")
    except Error as e:
        print(e)
    
    conn.close()
    
    return df_fromDB

In [5]:
# CUUR0000SA0: 
# CUSR0000SA0
seriesIDs = ["CUUR0000SA0", "CUSR0000SA0"] 
message, json_data = get_BLS_data(seriesIDs, 2004, 2023)
if message:
    print(message) 

2023, 2023, 


In [6]:
json_data

{'status': 'REQUEST_SUCCEEDED',
 'responseTime': 593,
 'message': ['No Data Available for Series CUUR0000SA0 Year: 2023',
  'No Data Available for Series CUSR0000SA0 Year: 2023'],
 'Results': {'series': [{'seriesID': 'CUUR0000SA0',
    'catalog': {'series_title': 'All items in U.S. city average, all urban consumers, not seasonally adjusted',
     'series_id': 'CUUR0000SA0',
     'seasonality': 'Not Seasonally Adjusted',
     'survey_name': 'CPI for All Urban Consumers (CPI-U)',
     'survey_abbreviation': 'CU',
     'measure_data_type': 'All items',
     'area': 'U.S. city average',
     'item': 'All items'},
    'data': [{'year': '2022',
      'period': 'M12',
      'periodName': 'December',
      'value': '296.797',
      'footnotes': [{}]},
     {'year': '2022',
      'period': 'M11',
      'periodName': 'November',
      'value': '297.711',
      'footnotes': [{}]},
     {'year': '2022',
      'period': 'M10',
      'periodName': 'October',
      'value': '298.012',
      'footnote

In [7]:
# Get number of series
number_of_series = len(json_data['Results']['series'])

In [8]:
df = pd.DataFrame(json_data['Results']['series'][0]['data'])
df

,year,period,periodName,value,footnotes
0,2022,M12,December,296.797,[{}]
1,2022,M11,November,297.711,[{}]
2,2022,M10,October,298.012,[{}]
3,2022,M09,September,296.808,[{}]
4,2022,M08,August,296.171,[{}]
...,...,...,...,...,...
223,2004,M05,May,189.1,[{}]
224,2004,M04,April,188.0,[{}]
225,2004,M03,March,187.4,[{}]
226,2004,M02,February,186.2,[{}]


In [9]:
df['month'] = df['period'].str.replace('M', '')
df['date'] = df['year'] + df['month']
df = df.sort_values(by=['date'], ignore_index=True)
df['date'] = pd.to_numeric(df['date'])
df = df.set_index(df["date"])
df = df.drop(columns=['year', 'period', 'periodName', 'footnotes', 'month', 'date'])
df

,value
date,
200401,185.2
200402,186.2
200403,187.4
200404,188.0
200405,189.1
...,...
202208,296.171
202209,296.808
202210,298.012


In [10]:
series_title = json_data['Results']['series'][0]['catalog']['series_title']
series_id = json_data['Results']['series'][0]['catalog']['series_id']
seasonality = json_data['Results']['series'][0]['catalog']['seasonality']
survey_long_name = json_data['Results']['series'][0]['catalog']['survey_name']
survey_short_name = json_data['Results']['series'][0]['catalog']['survey_name'][-6:-1]
survey_abbreviation = json_data['Results']['series'][0]['catalog']['survey_abbreviation']
measure_data_type = json_data['Results']['series'][0]['catalog']['measure_data_type']
area = json_data['Results']['series'][0]['catalog']['area']
item = json_data['Results']['series'][0]['catalog']['item']

In [11]:
create_table(series_id, db_name)
add_data_to_db(df, series_id, db_name)
check_df = table_to_df(series_id, db_name)
check_df

near ")": syntax error
Data added to DB: 
           value
date           
200401    185.2
200402    186.2
200403    187.4
200404    188.0
200405    189.1
...         ...
202208  296.171
202209  296.808
202210  298.012
202211  297.711
202212  296.797

[228 rows x 1 columns]


,value
date,
200401,185.2
200402,186.2
200403,187.4
200404,188.0
200405,189.1
...,...
202208,296.171
202209,296.808
202210,298.012


## Validate Data to Put into Correct Table

In [ ]:
 

    
def create_insert_trigger(tbl_name, db_name):
    '''
    Enter data into the log table when enter data into the database where the value does not exist or does not
    match the value that already exists
    '''
    conn = create_connection(db_name)
    c = conn.cursor()
    
    sql_code = """CREATE TRIGGER log_value_after_insert 
                    AFTER INSERT ON """ + seriesID + """ 
                    BEGIN INSERT INTO value_logs(      
                        seriesID,
                        year, 
                        period, 
                        periodName, 
                        value, 
                        date, 
                        date_action,
                        action)
                    VALUES(
                        new.seriesID,
                        new.year,
                        new.period,
                        new.periodName,
                        new.value,
                        new.date,
                        datetime('now','localtime'),
                        'INSERT'    
                    );
                    END;""".format(seriesID)
    
    conn_execute((sql_code)
    
    conn.close()    

def create_update_trigger(tbl_name, db_name):
    '''
    Enter data into the log table when enter data into the database where the value does not exist or does not
    match the value that already exists
    '''
    conn = create_connection(db_name)
    c = conn.cursor()
    
    sql_code = """CREATE TRIGGER log_value_after_update 
                    AFTER UPDATE ON """ + seriesID + """
                    WHEN old.value <> new.value
                    BEGIN INSERT INTO value_logs(      
                        seriesID,
                        year, 
                        period, 
                        periodName, 
                        value, 
                        date, 
                        date_action,
                        action)
                    VALUES(
                        new.seriesID,
                        new.year,
                        new.period,
                        new.periodName,
                        new.value,
                        new.date,
                        datetime('now','localtime'),
                        'INSERT'    
                    );
                    END;""".format(seriesID)
    conn_execute((sql_code)
    
    conn.close()

'''def unique_index(tbl_name, db_name)
    """
    Create a unique index for the date value of the table
    """
    conn = create_connection(db_name)
    c = conn.cursor()
    
    sql_code = 'CREATE UNIQUE INDEX idx_date ON ' + tbl_name + '(date)';
    
    conn_execute(sql_code)'''
    
def validate_df(df, tbl_name, db_name):
    """
    check if the data from the api df already exists in the table and 
    if the data in the table is duplicate or inconsistent
    return dataframe df without duplicate data or inconsistent data
    return dataframe df_dup_data with duplicate date
    return dataframe df_inconsistent_data with inconsistent data
    """
    conn = create_connection(db_name)
    c = conn.cursor()
    
    # Set up empty dataframes to use to store data temporarily
    df_not_dup_data = pd.DataFrame(columns = list(df.columns))
    df_dup_data = pd.DataFrame(columns = list(df.columns))
    df_inconsistent_data = pd.DataFrame(columns = list(df.columns))
    
    if check_table_exists(seriesID, db_name) != 0:
        # Check if data already exists in DB
        for i, row in df.iterrows():
            c.execute("SELECT date, value FROM " + tbl_name + " WHERE date = ?;", (df.loc[i]["date"],))
            queryOut = c.fetchall()
            
            # Data does not already exist in the database 
            if len(queryOut) == 0: 
                df_not_dup_data = pd.concat([df_not_dup_data, df.loc[[i]]])
            # Data already exists in the database with duplicate values
            elif queryOut[0][1] == df.loc[i]["value"]: 
                df_dup_data = pd.concat([df_dup_data, df.loc[[i]]])
            # Data already exists in the database but with different values
            elif queryOut[0][1] != df.loc[i]["value"]:
                df_inconsistent_data = pd.concat([df_inconsistent_data, df.loc[[i]]])            
    else:
        df_not_dup_data = df.copy()
    
    conn.close()
    
    df_not_dup_data = df_not_dup_data.sort_values(by=['date'])
    
    return df_not_dup_data, df_dup_data, df_inconsistent_data



    
    
def get_max_index(tbl_name, db_name):
    '''
    Get the maximum index value of the specified tbl_name
    '''
    conn = create_connection(db_name)
    c = conn.cursor()
    try:
        c.execute("SELECT MAX([index]) FROM " + seriesID)
        max_index = c.fetchone()[0]
        return max_index
    except Error as e:
        print(e)
    finally:
        conn.close()
        
def get_max_date(tbl_name, db_name):
    '''
    Get the maximum date from the specified tbl_name
    '''
    conn = create_connection(db_name)
    c = conn.cursor()
    try:
        c.execute("SELECT date FROM " + seriesID + " WHERE [index] = (SELECT MAX([index]) FROM " + seriesID +")")
        max_date = c.fetchone()[0]
        return max_date
    except Error as e:
        print(e)
    finally:
        conn.close()




